In [3]:
import pyrogue.utilities.fileio as fileio
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import struct
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from scipy.fftpack import fft
from scipy.signal import get_window
import pickle

# Abort

In [305]:
os.listdir('/mnt/SBOR/2024c_RFSoC/ZCU111/Abort/')

['LERFUJI_2024-10-17_01-05-01.npz',
 'LERFUJI_2024-10-10_22-11-36.npz',
 'LERFUJI_2024-10-15_11-30-38.npz',
 'LERFUJI_2024-10-10_21-29-54.npz',
 'LERFUJI_2024-10-13_11-42-07.npz',
 'LERFUJI_2024-10-17_09-02-58.npz',
 'LERFUJI_2024-10-13_12-47-29.npz',
 'LERFUJI_2024-10-16_01-06-09.npz',
 'LERFUJI_2024-10-14_00-14-04.npz',
 'LERFUJI_2024-10-15_21-28-08.npz',
 'LERFUJI_2024-10-16_08-57-48.npz',
 'LERFUJI_2024-10-13_14-34-59.npz',
 'LERFUJI_2024-10-14_11-04-39.npz',
 'LERFUJI_2024-10-17_00-58-37.npz',
 'LERFUJI_2024-10-16_07-17-11.npz',
 'LERFUJI_2024-10-15_23-43-13.npz',
 'LERFUJI_2024-10-14_00-35-00.npz',
 'LERFUJI_2024-10-15_20-22-59.npz',
 'LERFUJI_2024-10-15_10-14-14.npz',
 'LERFUJI_2024-10-10_21-12-23.npz',
 'LERFUJI_2024-10-16_04-52-14.npz',
 'LERFUJI_2024-10-15_20-56-49.npz',
 'LERFUJI_2024-10-13_00-10-19.npz',
 'LERFUJI_2024-10-14_01-44-58.npz',
 'LERFUJI_2024-10-15_19-08-53.npz',
 'LERFUJI_2024-10-15_10-21-29.npz',
 'LERFUJI_2024-10-15_08-37-16.npz',
 'LERFUJI_2024-10-13_03-50-4

In [22]:
def make_mastdplot(filename):
    npz_kw = np.load('/mnt/SBOR/2024c_RFSoC/ZCU111/Abort/'+filename,allow_pickle=True)
    X_sum=npz_kw['X_sum']
    X_delta=npz_kw['X_delta']
    Y_sum=npz_kw['Y_sum']
    Y_delta=npz_kw['Y_delta']
    X_pos=npz_kw['X_Pos']
    X_ma=npz_kw['X_MA']
    X_std=npz_kw['X_STD']
    #X_valid=npz_kw['X_valid']
    Y_pos=npz_kw['Y_Pos']
    Y_ma=npz_kw['Y_MA']
    Y_std=npz_kw['Y_STD']
    #Y_valid=npz_kw['Y_valid']
    #iv=npz_kw['Injection_Veto']
    abortflag=npz_kw['AbortFlag']


    if filename=="LERFUJI_2024-10-16_08-57-48.npz":
        X_std=np.zeros(len(X_pos))
        X_delta=np.zeros(len(X_sum))
    if filename=="LERFUJI_2024-10-15_19-08-53.npz":
        X_sum=np.zeros(len(X_delta))
    if filename=="LERFUJI_2024-10-16_01-58-36.npz":
        Y_sum=np.zeros(len(Y_delta))


    ##################    

    #find first abort gap
    start=0
    for i in range(0,2560):
        if i==2560:
            print("data is invalid")
            return
        if max(X_sum[i:i+50])<500:
            start=i+50
            if max(X_sum[i:i+70])>500:
                start=i+69
                break
            
    # determine the number of turns recorded in this file
    end=0
    for i in range(0,205):
        if start+(i+1)*2560 > len(X_sum):
            print('Data is invalid')
            return
        if np.sum(X_sum[start+i*2560:start+(i+1)*2560]) < 25600:
            end=i
            break
    if end%2==1:
        start=start+2560
        turn=end//2
    elif end%2==0:
        turn=end//2

    print(turn)

    threshold=np.std(X_sum)*0.5

    xposition=[]
    a=np.where(A>threshold, B/A, np.nan)
    
    for i in range(turn):
        A=X_sum[start+5120*i:start+5120*(i+1)]
        B=X_delta[start+5120*i:start+5120*(i+1)]
        turn_xpos=np.where(A>threshold, B/A, 0)
        xposition.append(np.where(turn_xpos<0.2, turn_xpos, 0))
        

    xposition=np.array(xposition)
    xpos=xposition[10:]-xposition[:-10]
    """
    print(xpos.shape)
    plt.figure(figsize=(10,10))
    plt.imshow(xpos,aspect='auto')
    plt.show()
    """

    
    #------------------
    if len(xpos)<10:
        return
    signal = np.concatenate(xpos)
    timesteps=np.arange(len(signal))/509
    """
    if len(signal)==0:
        return 0,0,0,0,[0],[0]
    fft_result = np.fft.fft(signal)
    fft_freqs = np.fft.fftfreq(len(signal),d=1/509)
    # 正の周波数部分のみ抽出
    positive_freqs = fft_freqs[:len(fft_freqs)//2]
    positive_fft = np.abs(fft_result[:len(fft_result)//2])
    """

    plt.figure(figsize=(18,4))
    plt.plot(timesteps,signal)
    plt.grid()
    plt.ylim(-0.4,0.4)
    #plt.xlim(,0)
    plt.savefig(f'abort_picture_2/{filename[8:-4]}_xpos.png',dpi=100)
    plt.close()
    
    #############################################
    """
    plt.figure(figsize=(16,4))
    plt.plot(positive_freqs, positive_fft)
    plt.title('FFT of Xpos')
    plt.xlabel('Frequency')
    plt.ylabel('Amplitude')
    plt.grid(True)
    #plt.xscale('log')
    #plt.savefig(f'abort_picture/{filename[8:-4]}_fft.png',dpi=100)
    plt.show()
    """




   # サンプリング周波数
    Fs = 509
    # ウィンドウサイズとシフトサイズ（少しずつ動かす）
    window_size = 5120
    step_size = 640
    # データの準備
    data = signal
    x = np.arange(len(data))/509
    # 窓関数（ハニング窓などを使用する）
    window = get_window('hann', window_size)
    # FFT結果を保存するリスト
    fft_results = []
    # ウィンドウの中心時間を保存するリスト
    window_times = []
    # データをウィンドウに分割してFFT
    for start in range(0, len(data) - window_size, step_size):
        # ウィンドウデータの抽出
        window_data = data[start:start + window_size]#* window
        # FFTの計算
        fft_result = np.abs(fft(window_data))[:window_size // 2]  # FFTの前半だけ使う（対称性を利用）
        # 周波数軸の計算
        freqs = np.fft.fftfreq(window_size, d=1/Fs)[:window_size // 2]
        # FFT結果の保存
        fft_results.append(fft_result)
        # ウィンドウの中心時間を計算して保存
        window_center_time = x[start + window_size]
        window_times.append(window_center_time)
    # ヒートマップ用にデータ整形
    fft_results = np.array(fft_results)

    
    #fft_results = np.log(fft_results + 1e-10)
    plt.figure(figsize=(16, 8))
    plt.imshow(fft_results.T, cmap='magma', aspect='auto',
               vmin=0,vmax=4,
              extent=(window_times[0]-window_times[-1]-10,-10,freqs[-1],freqs[0]))  # Y軸のラベル数を減らす
    plt.colorbar(label='Amplitude')
    # 軸ラベル
    plt.xlabel('Time [us]')
    plt.ylabel('Frequency [MHz]')
    plt.title('Time-varying FFT Heatmap Xpos')

    #plt.ylim(240,254)
    plt.savefig(f'abort_picture_2/{filename[8:-4]}_fftheatmapx.png',dpi=100)
    plt.close()




In [13]:
make_mastdplot('LERFUJI_2024-10-23_03-16-57.npz')

101


In [23]:

for i in os.listdir('/mnt/SBOR/2024c_RFSoC/ZCU111/Abort/'):
    print(i)
    make_mastdplot(i)
    

LERFUJI_2024-10-22_06-38-26.npz
101
LERFUJI_2024-10-19_01-22-42.npz
101
LERFUJI_2024-10-17_01-05-01.npz
0
LERFUJI_2024-10-22_15-14-57.npz
0
LERFUJI_2024-10-10_22-11-36.npz
101


/tmp/ipykernel_31477/2189317841.py:65: RuntimeWarning: divide by zero encountered in divide
  turn_xpos=np.where(A>threshold, B/A, 0)


LERFUJI_2024-10-22_03-20-25.npz
101
LERFUJI_2024-10-21_22-10-13.npz
0
LERFUJI_2024-10-15_11-30-38.npz
101
LERFUJI_2024-10-10_21-29-54.npz
101
LERFUJI_2024-10-21_21-15-49.npz
0
LERFUJI_2024-10-21_07-54-25.npz
101
LERFUJI_2024-10-22_11-57-54.npz
0
LERFUJI_2024-10-20_11-00-25.npz
101
LERFUJI_2024-10-21_21-04-24.npz
0
LERFUJI_2024-10-13_11-42-07.npz
101
LERFUJI_2024-10-22_12-04-43.npz
0
LERFUJI_2024-10-17_09-02-58.npz
101
LERFUJI_2024-10-21_23-02-15.npz
0
LERFUJI_2024-10-21_21-02-19.npz
0
LERFUJI_2024-10-13_12-47-29.npz
101
LERFUJI_2024-10-18_10-04-59.npz
Data is invalid
LERFUJI_2024-10-20_03-10-50.npz
101
LERFUJI_2024-10-22_03-00-16.npz
101
LERFUJI_2024-10-16_01-06-09.npz
92
LERFUJI_2024-10-21_21-19-37.npz
0
LERFUJI_2024-10-21_22-27-27.npz
0
LERFUJI_2024-10-21_22-28-41.npz
0
LERFUJI_2024-10-19_09-30-04.npz
101
LERFUJI_2024-10-14_00-14-04.npz
101
LERFUJI_2024-10-15_21-28-08.npz
95
LERFUJI_2024-10-20_13-51-29.npz
101
LERFUJI_2024-10-19_02-46-12.npz
101
LERFUJI_2024-10-16_08-57-48.npz
Data i